In [1]:
import json
import pandas as pd

In [2]:
conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v0/without_head/conversations.json'
graph_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v0/without_head/graph.jsonl'

In [3]:
import re

def find_module_string(text):
    # Regex pattern to find the string starting with "module" and ending with ";"
    # It uses non-greedy matching to find the shortest match
    pattern = r"module.*?;"
    
    # Using re.DOTALL to make the dot match all characters including newline
    match = re.search(pattern, text, re.DOTALL)
    
    if match:
        return match.group()  # Returns the matched string
    else:
        return None  # No match found

# Example usage
input_text = """
some initial text
module name="example1" \n   attribu\nte="   value";
some other text
module name="example2" attribute="another value";
"""

# Find the first occurrence
result = find_module_string(input_text)
print("Found substring:", result)

# If you need to find all occurrences instead of just the first one, you can use re.findall
all_matches = re.findall(r"module.*?;", input_text, re.DOTALL)
print("All found substrings:", all_matches)

Found substring: module name="example1" 
   attribu
te="   value";
All found substrings: ['module name="example1" \n   attribu\nte="   value";', 'module name="example2" attribute="another value";']


In [4]:
graph = pd.read_json(graph_path, lines=True)
with open(conv_path, 'r') as f:
    conv = json.load(f)


In [5]:
conv[0]['conversations'][0]['value']

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'instr', 'type': 'input port'}, {'id': 1, 'content': 'op_type', 'type': 'output port'}, {'id': 2, 'content': 'alu_op', 'type': 'output port'}, {'id': 3, 'content': 'mem_control', 'type': 'output port'}, {'id': 4, 'content': 'branch_taken', 'type': 'output port'}, {'id': 5, 'content': 'arith_dec', 'type': 'submodule'}, {'id': 6, 'content': 'mem_dec', 'type': 'submodule'}, {'id': 7, 'content': 'branch_dec', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 0, 0, 5, 5, 6, 7], [5, 6, 7, 1, 2, 3, 4]]\n, Design a RISC-V instruction decoder module that decodes 32-bit instructions into multiple control signals for further execution stages. The module identifies the type of operation (arithmetic, memory, branch) and specific control flags (e.g., ALU operation, memory read/write, branch taken).\n\nModule name:\n    instr_decoder\n\nInput ports:\n    instr [31:0]: Full 32-bit instruction input.\n\nO

In [6]:
conv[0]['conversations'][1]['value']

"module instr_decoder (\n    input [31:0] instr,\n    output [2:0] op_type,\n    output [3:0] alu_op,\n    output [1:0] mem_control,\n    output branch_taken\n);\n\n    wire [6:0] opcode;\n    wire [2:0] funct3;\n    wire [6:0] funct7;\n\n    assign opcode = instr[6:0];\n    assign funct3 = instr[14:12];\n    assign funct7 = instr[31:25];\n\n    // Submodules for decoding different instruction fields\n    arithmetic_decoder arith_dec (\n        .opcode(opcode),\n        .funct3(funct3),\n        .funct7(funct7),\n        .op_type(op_type),\n        .alu_op(alu_op)\n    );\n\n    memory_decoder mem_dec (\n        .opcode(opcode),\n        .funct3(funct3),\n        .mem_control(mem_control)\n    );\n\n    branch_decoder branch_dec (\n        .opcode(opcode),\n        .branch_taken(branch_taken)\n    );\n\nendmodule\n\nmodule arithmetic_decoder (\n    input [6:0] opcode,\n    input [2:0] funct3,\n    input [6:0] funct7,\n    output reg [2:0] op_type,\n    output reg [3:0] alu_op\n);\n\n  

In [7]:
head = find_module_string(conv[0]['conversations'][1]['value'])
conv[0]['conversations'][0]['value'] + head

"Given a submodules interconnection graph: \n<graph>\n, \nnodes: [{'id': 0, 'content': 'instr', 'type': 'input port'}, {'id': 1, 'content': 'op_type', 'type': 'output port'}, {'id': 2, 'content': 'alu_op', 'type': 'output port'}, {'id': 3, 'content': 'mem_control', 'type': 'output port'}, {'id': 4, 'content': 'branch_taken', 'type': 'output port'}, {'id': 5, 'content': 'arith_dec', 'type': 'submodule'}, {'id': 6, 'content': 'mem_dec', 'type': 'submodule'}, {'id': 7, 'content': 'branch_dec', 'type': 'submodule'}], \nedge_attrs: [], \nconnectivity: [[0, 0, 0, 5, 5, 6, 7], [5, 6, 7, 1, 2, 3, 4]]\n, Design a RISC-V instruction decoder module that decodes 32-bit instructions into multiple control signals for further execution stages. The module identifies the type of operation (arithmetic, memory, branch) and specific control flags (e.g., ALU operation, memory read/write, branch taken).\n\nModule name:\n    instr_decoder\n\nInput ports:\n    instr [31:0]: Full 32-bit instruction input.\n\nO

In [8]:
for i in range(len(conv)):
    module_head = find_module_string(conv[i]['conversations'][1]['value'])
    conv[i]['conversations'][0]['value'] = conv[i]['conversations'][0]['value'] + '\n' + module_head

In [ ]:
# conv[100]['conversations'][0]['value']

IndexError: list index out of range

In [10]:
new_conv_path = '/data/LPJ/ICML25/GraphCoder/graphgpt_dataset/gpt_dataset_construction/more200_gpt4/specific_task/instr_reg/v0/with_head/conversations.json'



In [11]:
with open(new_conv_path, 'w') as f:
    json.dump(conv, f)

In [12]:
import json
with open(new_conv_path, 'r') as f:
    new_conv = json.load(f)
    


In [13]:
len(new_conv)

480

In [ ]:
new_conv[100]['conversations'][0]['value']

In [32]:
len(graph)

2148